# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = pd.DataFrame(data.groupby(['CustomerID', 'ProductName']).Quantity.sum())
df = df.reset_index()

In [5]:
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
product_by_customer = df.pivot_table(index=["CustomerID"], columns=["ProductName"], values=["Quantity"]).fillna(0)
#instead of doing transpose, already assign productName to columns
product_by_customer

Quantity                                    \
ProductName Anchovy Paste - 56 G Tube Appetizer - Mini Egg Roll, Shrimp   
CustomerID                                                                
33                                0.0                               0.0   
200                               0.0                               0.0   
264                               0.0                               0.0   
356                               0.0                               0.0   
412                               0.0                               0.0   
...                               ...                               ...   
97928                             0.0                               0.0   
98069                             0.0                               0.0   
98159                             0.0                               0.0   
98185                             0.0                               0.0   
98200                             0.0                               0.0   

                                                                 \
ProductName Appetizer - Mushroom Tart Appetizer - Sausage Rolls   
CustomerID                                                        
33                                0.0                       0.0   
200                               0.0                       0.0   
264                               0.0                       0.0   
356                               0.0                       0.0   
412                               0.0                       0.0   
...                               ...                       ...   
97928                             0.0                      25.0   
98069                             0.0                      25.0   
98159                             0.0                       0.0   
98185                            25.0                      25.0   
98200                             0.0                       0.0   

                                                               \
ProductName Apricots - Dried Apricots - Halves Apricots Fresh   
CustomerID                                                      
33                       1.0               0.0            0.0   
200                      0.0               0.0            0.0   
264                      0.0               1.0            1.0   
356                      0.0               0.0            0.0   
412                      1.0               0.0            0.0   
...                      ...               ...            ...   
97928                    0.0              50.0            0.0   
98069                    0.0              25.0            0.0   
98159                    0.0               0.0            0.0   
98185                    0.0              25.0            0.0   
98200                    0.0              25.0           25.0   

                                                                          ...  \
ProductName Arizona - Green Tea Artichokes - Jerusalem Assorted Desserts  ...   
CustomerID                                                                ...   
33                          0.0                    0.0               1.0  ...   
200                         0.0                    0.0               0.0  ...   
264                         0.0                    0.0               0.0  ...   
356                         0.0                    0.0               0.0  ...   
412                         0.0                    0.0               0.0  ...   
...                         ...                    ...               ...  ...   
97928                      25.0                    0.0               0.0  ...   
98069                       0.0                    0.0              25.0  ...   
98159                       0.0                    0.0               0.0  ...   
98185                       0.0                    0.0               0.0  ...   
98200                       0.0                    0.0              25.0  .

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
#could also normalize data before, now working on not normalized

def customer_dist(summary, metric='euclidean'):
    pairwise = pd.DataFrame(
        1/(1 + squareform(pdist(summary, metric))),
        columns = summary.index,
        index = summary.index
    )
    return pairwise

In [8]:
euclidean_dist = customer_dist(product_by_customer)
euclidean_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [9]:
similar_customers = list(euclidean_dist[33].sort_values(ascending=False)[1:].head().index)
similar_customers
#returning the list that containt customer IDs becuase matrix is done from IDs

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [10]:
similar_customer_records = df[df.CustomerID.isin(similar_customers)]
similar_customer_records
#from df taking only customers with these IDs

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [11]:
grouped = pd.DataFrame(similar_customer_records.groupby('ProductName').Quantity.sum())
product_records = grouped.sort_values('Quantity', ascending=False).reset_index()
product_records

,ProductName,Quantity
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
2,Towels - Paper / Kraft,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3
...,...,...
213,Hinge W Undercut,1
214,Ice Cream Bar - Hageen Daz To,1
215,Jagermeister,1
216,Jolt Cola - Electric Blue,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [12]:
merged = pd.merge(product_records, pd.DataFrame(product_by_customer.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
customer_orders = merged[merged['Purchased']==0].head()
#filter where customer quanitity is 0
products_to_recommend = list(customer_orders['ProductName'].values)
products_to_recommend
#showing the values of the top 5 product recommendations for the customer id 33

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [13]:
all_recom = {}
unique_ids = list(data.CustomerID.unique())

for id in unique_ids:
    similar_customers = list(euclidean_dist[id].sort_values(ascending=False)[1:].head().index)
    similar_customer_records = df[df.CustomerID.isin(similar_customers)]
    grouped = pd.DataFrame(similar_customer_records.groupby('ProductName').Quantity.sum())
    product_records = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(product_records, pd.DataFrame(product_by_customer.T[id]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    customer_orders = merged[merged['Purchased']==0].head()
    #filter where customer quanitity is 0
    products_to_recommend = list(customer_orders['ProductName'].values)

    all_recom[id] = ({'product1': products_to_recommend[0], 'product2': products_to_recommend[1],
                      'product3': products_to_recommend[2], 'product4': products_to_recommend[3],
                      'product5': products_to_recommend[4]})
print(all_recom)

{61288: {'product1': 'Jagermeister', 'product2': 'Chicken - Soup Base', 'product3': 'Lime Cordial - Roses', 'product4': 'Macaroons - Two Bite Choc', 'product5': 'Flavouring - Orange'}, 77352: {'product1': 'Wine - Valpolicella Masi', 'product2': 'Tahini Paste', 'product3': 'Tuna - Salad Premix', 'product4': 'Chicken - Soup Base', 'product5': 'Isomalt'}, 40094: {'product1': 'Bread - Italian Corn Meal Poly', 'product2': 'Puree - Mocha', 'product3': 'Tuna - Salad Premix', 'product4': 'Sherry - Dry', 'product5': 'Beer - Sleemans Cream Ale'}, 23548: {'product1': 'Squid - Tubes / Tenticles 10/20', 'product2': 'Sprouts - Baby Pea Tendrils', 'product3': 'Flavouring - Orange', 'product4': 'Bread - Raisin Walnut Oval', 'product5': 'Banana Turning'}, 78981: {'product1': 'Vanilla Beans', 'product2': 'Lettuce - Frisee', 'product3': 'Yogurt - Blueberry, 175 Gr', 'product4': 'Cinnamon Buns Sticky', 'product5': 'Pop Shoppe Cream Soda'}, 83106: {'product1': 'Cocoa Butter', 'product2': 'Cheese - Boursin,

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [14]:
recommended_products_euc = pd.DataFrame(all_recom)
recommended_products_euc = recommended_products_euc.T.reset_index().rename(columns={'index': 'id'})
recommended_products_euc

,id,product1,product2,product3,product4,product5
0,61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
3,23548,Squid - Tubes / Tenticles 10/20,Sprouts - Baby Pea Tendrils,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
...,...,...,...,...,...,...
995,49005,"Pasta - Penne, Rigate, Dry",Browning Caramel Glace,Tea - Decaf Lipton,"Garlic - Primerba, Paste","Wine - Red, Harrow Estates, Cab"
996,41286,Veal - Inside,"Thyme - Lemon, Fresh",Wonton Wrappers,"Veal - Inside, Choice",Bread Crumbs - Japanese Style
997,85878,Olives - Kalamata,Wine - White Cab Sauv.on,Veal - Eye Of Round,Bread - Calabrese Baguette,Cheese - Wine
998,68506,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,"Rosemary - Primerba, Paste",Scallops - 10/20,"Oregano - Dry, Rubbed"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [15]:
def customer_dist(summary, metric='cosine'):
    pairwise = pd.DataFrame(
        1/(1 + squareform(pdist(summary, metric))),
        columns = summary.index,
        index = summary.index
    )
    return pairwise

In [16]:
cosine_dist = customer_dist(product_by_customer)
cosine_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.513809,0.521925,0.560225,0.530406,0.536686,0.542873,0.540381,0.533190,0.535224,0.526495,...,0.532674,0.539309,0.522883,0.542570,0.569539,1.000000,0.562050,0.543657,0.531363,0.519127
98069,0.528795,0.573177,0.548538,0.520138,0.519628,0.520835,0.534221,0.525789,0.531918,0.541402,...,0.529233,0.535979,0.530224,0.518742,0.554911,0.562050,1.000000,0.548011,0.542143,0.536728
98159,0.520208,0.537284,0.538340,0.536202,0.538710,0.533856,0.535601,0.535704,0.544926,0.535721,...,0.542777,0.556893,0.535487,0.521507,0.541404,0.543657,0.548011,1.000000,0.543258,0.531447


In [17]:
all_recom_cos = {}
unique_ids = list(data.CustomerID.unique())

for id in unique_ids:
    similar_customers = list(cosine_dist[id].sort_values(ascending=False)[1:].head().index)
    similar_customer_records = df[df.CustomerID.isin(similar_customers)]
    grouped = pd.DataFrame(similar_customer_records.groupby('ProductName').Quantity.sum())
    product_records = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(product_records, pd.DataFrame(product_by_customer.T[id]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    customer_orders = merged[merged['Purchased']==0].head()
    #filter where customer quanitity is 0
    products_to_recommend = list(customer_orders['ProductName'].values)

    all_recom_cos[id] = ({'product1': products_to_recommend[0], 'product2': products_to_recommend[1],
                      'product3': products_to_recommend[2], 'product4': products_to_recommend[3],
                      'product5': products_to_recommend[4]})
print(all_recom_cos)

{61288: {'product1': 'Jagermeister', 'product2': 'Wine - Two Oceans Cabernet', 'product3': 'Tofu - Firm', 'product4': 'Appetizer - Sausage Rolls', 'product5': 'Dc - Frozen Momji'}, 77352: {'product1': 'Bread - Rye', 'product2': 'Fondant - Icing', 'product3': 'Lettuce - Frisee', 'product4': 'Bread - French Baquette', 'product5': 'Pork - Kidney'}, 40094: {'product1': 'Juice - V8 Splash', 'product2': 'Beef - Inside Round', 'product3': 'Bread - Roll, Soft White Round', 'product4': 'Truffle Cups - Brown', 'product5': 'Cake - Mini Cheesecake'}, 23548: {'product1': 'Juice - Orange', 'product2': 'Bagel - Plain', 'product3': 'Olives - Stuffed', 'product4': 'Veal - Sweetbread', 'product5': 'Cinnamon Buns Sticky'}, 78981: {'product1': 'Wine - Blue Nun Qualitatswein', 'product2': 'Cookies - Assorted', 'product3': 'Cheese - Cottage Cheese', 'product4': 'Baking Powder', 'product5': 'Quiche Assorted'}, 83106: {'product1': 'Bouq All Italian - Primerba', 'product2': 'Onions - Vidalia', 'product3': 'Sca

In [18]:
recommended_products_cos = pd.DataFrame(all_recom_cos)
recommended_products_cos = recommended_products_cos.T.reset_index().rename(columns={'index': 'id'})
recommended_products_cos

,id,product1,product2,product3,product4,product5
0,61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,77352,Bread - Rye,Fondant - Icing,Lettuce - Frisee,Bread - French Baquette,Pork - Kidney
2,40094,Juice - V8 Splash,Beef - Inside Round,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Cake - Mini Cheesecake
3,23548,Juice - Orange,Bagel - Plain,Olives - Stuffed,Veal - Sweetbread,Cinnamon Buns Sticky
4,78981,Wine - Blue Nun Qualitatswein,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted
...,...,...,...,...,...,...
995,49005,Squid - Tubes / Tenticles 10/20,"Pepper - White, Ground",Soup Knorr Chili With Beans,Cheese - Bocconcini,Barramundi
996,41286,"Hickory Smoke, Liquid","Salsify, Organic",Dc Hikiage Hira Huba,Oil - Shortening - All - Purpose,Banana - Leaves
997,85878,Bread - Calabrese Baguette,French Pastry - Mini Chocolate,Cheese - Parmesan Grated,Foam Dinner Plate,Cheese - Mix
998,68506,Flavouring - Orange,Longos - Chicken Wings,"Cheese - Brie, Triple Creme",Tia Maria,Fondant - Icing
